In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imripy.merger_system as ms
from imripy import halo
from imripy import inspiral
from imripy import waveform
from imripy import detector

In [ ]:
m1 = 10.*ms.solar_mass_to_pc
m2 = 10.*ms.solar_mass_to_pc
D = 5e8

sp = ms.SystemProp(m1, m2, halo.ConstHalo(0.), D=D, inclination_angle=np.pi/2., pericenter_angle=np.pi/4.)

In [ ]:
# first, compare to Maggiore 2007
a0 = 500.* sp.r_isco()
afin = 1.*sp.r_isco()

e0 = 0.9

t_0, a_0, e_0 = inspiral.Classic.evolve_elliptic_binary(sp, a0, e0, afin, acc=1e-10)

In [ ]:
def g(e):
    return e**(12./19.)/(1. - e**2) * (1. + 121./304. * e**2)**(870./2299.)

plt.figure()
plt.plot(e_0, a_0, label='numeric')
plt.plot(e_0, a0 * g(e_0)/g(e0), label='analytic', linestyle='--')
plt.xlabel('e'); plt.ylabel('a')
plt.yscale('log')
plt.grid(); plt.legend()

In [ ]:
# Now compare to 1807.07163
a0 = 20.*sp.r_isco()
e0 = 0.6
t, a, e = inspiral.Classic.evolve_elliptic_binary(sp, a0, e0, a_fin = 1e-1*sp.r_isco(), acc=1e-13)

In [ ]:
fig, (ax_a, ax_e) = plt.subplots(2, 1, figsize=(8,10))
ax_a.plot(t/ms.year_to_pc, a)

ax_e.plot(t/ms.year_to_pc, e)

ax_a.grid(); ax_a.set_xscale('log'); ax_a.set_ylabel('y')
ax_e.grid(); ax_e.set_xscale('log'); ax_e.set_ylabel('e'); ax_e.set_xlabel('t / yr')

In [ ]:
n_comp = 20

wfs = [waveform.h_n(n, sp, t, a, e, acc=1e-13) for n in range(1, n_comp+1)]

In [ ]:
n_disp = 5
plt.figure(figsize=(10, 8))

for i in range( min(len(wfs), n_disp)):
    plt.plot(wfs[i][0]/ms.hz_to_invpc, np.abs(wfs[i][1]), label=r"$|h^{(" + str(i+1) + ")}_+|$")

f_gw = np.geomspace(np.min(wfs[0][0]), np.max(wfs[n_disp][0]), 5000)

h_plus_tot = np.sum([ np.interp(f_gw, wf[0], wf[1], left=0., right=0.) * np.exp(1.j * np.interp(f_gw, wf[0], wf[3], left=0., right=0.)) for wf in wfs ],  axis=0)
h_cross_tot = np.sum([ np.interp(f_gw, wf[0], wf[2], left=0., right=0.) * np.exp(1.j * np.interp(f_gw, wf[0], wf[3], left=0., right=0.)) for wf in wfs ],  axis=0)
plt.plot(f_gw/ms.hz_to_invpc, np.abs(h_plus_tot), label=r"$|h^{SPA}_+|$")

plt.xlim(left=np.min(wfs[0][0])/ms.hz_to_invpc, right=np.max(wfs[n_disp][0])*4e-3/ms.hz_to_invpc)
#plt.xscale('log')
plt.xlabel('f / Hz'); 
plt.grid(); plt.legend();

In [ ]:
plt.figure(figsize=(10, 8))
plt.loglog(f_gw/ms.hz_to_invpc, 2.*f_gw*np.abs(h_plus_tot), label=r"$|h^{SPA}_+|$")
#plt.loglog(f_gw, 2.*f_gw*np.abs(h_2_cross), label=r"$|h^{(2)}_x|$")

f = np.geomspace(detector.Lisa().Bandwith()[0], detector.Lisa().Bandwith()[1], 100)
plt.plot(f/ms.hz_to_invpc, detector.Lisa().NoiseStrain(f), label='LISA')

plt.ylim(1e-22, 2e-18)
#plt.xlim(detector.Lisa().Bandwith()[0]/ms.hz_to_invpc, detector.Lisa().Bandwith()[1]/ms.hz_to_invpc, )
plt.xlabel('f / Hz'); plt.ylabel('characteristic strain')
plt.grid(); plt.legend();

In [ ]:
t_plot = np.linspace(np.min(t) if t[0] > 0. else t[1]*1e-1, np.max(t), 500)
f_plot = np.linspace(np.min(f_gw), np.max(f_gw)/1000., 500)
t_plot, f_plot = np.meshgrid(t_plot, f_plot)

h_plus_plot = np.zeros(shape=np.shape(t_plot))
h_cross_plot = np.zeros(shape=np.shape(t_plot))
for i in range(len(t_plot[0])):
    for wf in wfs:
        #print(t_plot[i,0])
        f = np.interp(t_plot[0, i], t, wf[0], left=0., right=0.)
        index_f = (np.abs(f_plot[:, i] - f)).argmin()
        #print(f, f_plot[i], index_f)
        h_plus_plot[index_f, i] = np.abs(np.interp(f_plot[index_f, i], wf[0], wf[1]))
        h_cross_plot[index_f, i] = np.abs(np.interp(f_plot[index_f, i], wf[0], wf[2]))
        
h_plus_plot = h_plus_plot/np.max(h_plus_plot)

plt.figure(figsize=(10, 8))
#plt.xscale('log'); plt.yscale('log')
plt.contourf( t_plot/ms.s_to_pc, f_plot/ms.hz_to_invpc, h_plus_plot)

plt.figure(figsize=(10, 8))
#plt.xscale('log'); plt.yscale('log')
plt.contourf( t_plot/ms.s_to_pc, f_plot/ms.hz_to_invpc, h_cross_plot)

plt.show()